In [26]:
from dotenv import load_dotenv

load_dotenv()

True

In [27]:
import os
from typing import Literal

# Make sure to install the needed packages:
# pip install langgraph langgraph-supervisor langchain-openai

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import ToolNode

# --- Weather tool ---
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

# We'll create a model and bind the tool so the LLM knows it can call `get_weather`.
tools = [get_weather]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# --- Existing agent workflow definition ---
def call_model(state: MessagesState):
    """Call the LLM with the conversation so far."""
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    """If there's a tool call requested, go to 'tools', else end."""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

weather_workflow = StateGraph(MessagesState)

tool_node = ToolNode(tools)

weather_workflow.add_node("agent", call_model)
weather_workflow.add_node("tools", tool_node)

weather_workflow.add_edge(START, "agent")
weather_workflow.add_conditional_edges("agent", should_continue)
weather_workflow.add_edge("tools", "agent")

weather_agent_graph =  weather_workflow.compile(name="weather_agent")

In [28]:
from langgraph_supervisor import create_supervisor

supervisor_workflow = create_supervisor(
    agents=[weather_agent_graph],
    model=model,
    prompt=(
        "You are a supervisor managing a weather agent. "
        "For any weather-related question, call the 'weather_agent' to handle it."
    ),
    # output_mode="last_message",
    output_mode="full_history",
    supervisor_name="supervisor_agent",
)

supervisor_app = supervisor_workflow.compile()


In [29]:
supervisor_app.invoke(
    {"messages": [HumanMessage(content="Hello there, how are you?")]}
)

{'messages': [HumanMessage(content='Hello there, how are you?', additional_kwargs={}, response_metadata={}, id='a1cb2c04-7f46-4123-8308-3fb62c0f7621'),
  AIMessage(content="I'm just a program, so I don't have feelings, but I'm here and ready to help you! What can I assist you with today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 71, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, name='supervisor_agent', id='run-51f280bd-2c17-4560-b392-ce40ff49e229-0', usage_metadata={'input_tokens': 71, 'output_tokens': 30, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0

In [30]:
supervisor_app.invoke(
    {"messages": [HumanMessage(content="How is the weather in Munich?")]}
)


{'messages': [HumanMessage(content='How is the weather in Munich?', additional_kwargs={}, response_metadata={}, id='26a00b03-31f4-4303-a754-ea856338330a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dJL5BsZsqspsESQG5pJhWMRW', 'function': {'arguments': '{}', 'name': 'transfer_to_weather_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 71, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor_agent', id='run-ec7d8b7e-eebd-4adb-94a8-2d21589f988e-0', tool_calls=[{'name': 'transfer_to_weather_agent', 'args': {}, 'id': 'call_dJL5BsZsqspsESQG5pJhWMRW', 'type': 'tool_call'}], us